In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')

# visualize results module
from visualize import visualize_svm as Visualize



Autosaving every 180 seconds


<ipython-input-1-eeb477d8c4a5>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#####################################################
#####################################################
#####################################################
def get_lever_pulls_and_behaviors(session_id, root_dir, animal_id):

    # look at specific trials and neural activity
    fname = os.path.join(
        root_dir, 
        animal_id,
        'tif_files',
        session_id,
        session_id+'_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz')

    data = np.load(fname, allow_pickle=True)
    names_behaviors = data['names']
    #print ("names: ", names_behaviors)
    trials = data['trials']
    #print (trials.shape)
    random = data['random']
    behavior_times = data['behaviors']
    #print (behavior_times.shape)
    # t = np.arange(trials.shape[1])/30.-30
    # plt.plot(t,trials[2,:,:])
    # plt.show()

    shift = np.loadtxt(os.path.join(
                                root_dir, 
                                animal_id,
                                'tif_files',
                                session_id,
                                'shift.txt'))

    #print ("Shifts: ", shift)

    blue_light = np.load(os.path.join(
                                root_dir, 
                                animal_id,
                                'tif_files',
                                session_id,
                                'blue_light_frame_triggers.npz'))

    start_blue = blue_light['start_blue']/30
    #print ("start blue: ", start_blue)

    fname = os.path.join(
                    root_dir, 
                    animal_id,
                    'tif_files',
                    session_id,
                    session_id+'_locanmf_wholestack.npz')

    data = np.load(fname, allow_pickle=True)

    names = data['areanames_area']
    areas = data['areas']
    names2 = data['names']
    A = data["A_reshape"]
    #print (A.shape)
    whole_stack = data["whole_stack"]
    #print (whole_stack.shape)

    clrs = ['pink','blue','lightblue','slategrey','black','magenta','khaki']
    ctr_clrs =0
    #

    area_ids = [12, 8, 13]  # left limb, left M1, left visual

    # rewarded lever pulls in seconds
    reward_times_sec = behavior_times[0]/30. + start_blue

    # get left paw, right paw and licking
    feature_ids = [2,3,7]
    behavior_times_sec = behavior_times[feature_ids]/30 + start_blue

    return reward_times_sec, behavior_times_sec

In [13]:
###########################################
############ LOAD LOCANMF COMPS ###########
###########################################
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'
session_id = 'IA1pm_Feb2_30Hz'
session_id = 'IA1pm_Feb16_30Hz'
#session_id = 'IA1pm_Feb3_30Hz'

# animal_id = 'AQ2'
# session_id = 'AQ2am_Apr1_Week4_30Hz'


sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  'all')
print ("# sessions: ", len(sessions))

# look at specific trials and neural activity
fname = os.path.join(
    root_dir, 
    animal_id,
    'tif_files',
    session_id,
    session_id+'_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz')

data = np.load(fname, allow_pickle=True)
names_behaviors = data['names']
print ("names: ", names_behaviors)
trials = data['trials']
print (trials.shape)
random = data['random']
behavior_times = data['behaviors']
print (behavior_times.shape)
# t = np.arange(trials.shape[1])/30.-30
# plt.plot(t,trials[2,:,:])
# plt.show()

shift = np.loadtxt(os.path.join(
                            root_dir, 
                            animal_id,
                            'tif_files',
                            session_id,
                            'shift.txt'))
                   
print ("Shifts: ", shift)

blue_light = np.load(os.path.join(
                            root_dir, 
                            animal_id,
                            'tif_files',
                            session_id,
                            'blue_light_frame_triggers.npz'))

start_blue = blue_light['start_blue']/30
print ("start blue: ", start_blue)

fname = os.path.join(
                root_dir, 
                animal_id,
                'tif_files',
                session_id,
                session_id+'_locanmf_wholestack.npz')

data = np.load(fname, allow_pickle=True)

names = data['areanames_area']
areas = data['areas']
names2 = data['names']
A = data["A_reshape"]
print (A.shape)
whole_stack = data["whole_stack"]
print (whole_stack.shape)

clrs = ['pink','blue','lightblue','slategrey','black','magenta','khaki']
ctr_clrs =0
#
for k in range(len(names2)):
    print (k, names2[k])

area_ids = [12, 8, 13]  # left limb, left M1, left visual
ca = whole_stack[area_ids]
print (ca.shape)

#
plt.figure()
t = np.arange(ca.shape[1])/30.

# plot calcium activity
for k in range(ca.shape[0]):
    temp = ca[k]
    temp = temp - np.median(temp) + k*350
    plt.plot(t,temp, 
             c=clrs[ctr_clrs],
             linewidth=2,
             label = names2[area_ids][k])

    ctr_clrs+=1
    
# plot rewarded lever pulls
temp = behavior_times[0]/30.
plt.axvline(x=temp[0]
            #+shift[0]
            +start_blue
            ,linestyle='--',
            c=clrs[ctr_clrs],
           label='rewarded lever pull')
for k in range(1,temp.shape[0],):
    plt.axvline(x=temp[k]
                #+shift[0]
                +start_blue
            ,linestyle='--',

                c=clrs[ctr_clrs])
ctr_clrs+=1
    
# plot rest of events
ctr=0
for k in [2,3,7]:
    temp = behavior_times[k]
    print (temp.shape)
    temp = temp/30+ start_blue
    plt.scatter(temp , 
                temp*0+1300+ ctr*50, 
                c=clrs[ctr_clrs],
                label=names_behaviors[k])
    ctr+=1
    ctr_clrs+=1
    
plt.legend()
plt.show()



# sessions:  71
names:  ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
 'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
(42, 1800, 16)
(9,)
Shifts:  [4.641 4.915 4.701]
start blue:  2.5
(128, 128, 16)
(16, 40000)
0 Retrosplenial area, dorsal part, layer 1 - right
1 Retrosplenial area, lateral agranular part, layer 1 - right
2 Primary visual area, layer 1 - right
3 Primary somatosensory area, upper limb, layer 1 - right
4 Primary somatosensory area, lower limb, layer 1 - right
5 Primary somatosensory area, barrel field, layer 1 - right
6 Secondary motor area, layer 1 - right
7 Primary motor area, Layer 1 - right
8 Primary motor area, Layer 1 - left
9 Secondary motor area, layer 1 - left
10 Primary somatosensory area, barrel field, layer 1 - left
11 Primary somatosensory area, lower limb, layer 1 - left
12 Primary somatosensory area, upper limb, layer 1 - left
13 Primary visual area, layer 1 - left
14 Retrosplenial area, lateral agranular part, layer 1 - left
15 R

In [70]:
################################################
########## MAKE BEHAVIOR SEQUENCES IMAGE #######
################################################
#
from tqdm import tqdm

#
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'
session_id_target = 'IA1pm_Feb16_30Hz'


# 
sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  'all')
print ("# sessions: ", len(sessions))

allstacks = []
#
sums_array = [[],[],[]]
for session_id in tqdm(sessions):
    
    #if session_id != session_id_target:
    #    continue
    
    try:
        reward_times_sec, behavior_times_sec = get_lever_pulls_and_behaviors(session_id, root_dir, animal_id)
    except:
        continue

    # make behavivor boolean vectors
    bin_width = 0.250 # bin width in seconds
    window = [-15,5]
    #behavior_binned = np.zeros((int(10*1//bin_width)))
    behavior_binned = np.zeros((int((window[1]-window[0])*1//bin_width)))
    
    # 
    names = ['left paw','right paw','licking']
    
    # loop over lever pulls:
    behaviors = [0,1,2]
    stacks = []
    for behavior in behaviors: 
        vector_stack = []
        for k in range(reward_times_sec.shape[0]):
            temp1 = reward_times_sec[k]
            temp_behavior = behavior_binned.copy()
            for p in range(behavior_times_sec[behavior].shape[0]):
                temp2 = behavior_times_sec[behavior][p]
                diff = temp1-temp2
                if diff<=15 and diff>-5:
                    # find bin where it belongs
                    idx = int(behavior_binned.shape[0] - diff/bin_width)
                    #print (diff, idx)
                    idx -= int(window[1]/bin_width)
                    #if idx==(window[1]-window[0])/bin_width:
                    #   idx-=1
                    temp_behavior[idx] = 1

            vector_stack.append(temp_behavior)

        vector_stack = np.array(vector_stack)
        #print (vector_stack.shape)

        #
        if True:
                sums = np.sum(vector_stack,
                              axis=0)
                #sums_norm = sums/np.max(sums)
                sums = sums/vector_stack.shape[0]
                sums_array[behavior].append(sums)
                
        #
        if False:
            clrs1 = ['Greys','Purples','copper']

            plt.figure(figsize=(10,5))
            clb = plt.imshow(vector_stack,
                             aspect='auto',
                             extent=[window[0],window[1],0,vector_stack.shape[0]],
                             cmap=clrs1[behavior]
                            )
            plt.colorbar()
            plt.xlabel("Time (sec)")
            plt.ylabel("Trial #")

            
            if True:
                sums = np.sum(vector_stack,axis=0)
                sums_array.append(sums)
                t=np.arange(window[0], window[1],bin_width)
                plt.plot(t, sums)
            
            
            plt.savefig('/home/cat/'+str(behavior)+'.svg')
            plt.close()

        #
        if False:
            clrs1 = ['Greys','Purples','copper']
            plt.figure(figsize=(10,5))
            clb = plt.imshow(vector_stack,
                             aspect='auto',
                             extent=[-10,0,0,vector_stack.shape[0]],
                             cmap=clrs1[behavior]
                            )
            plt.colorbar()
            plt.xlabel("Time (sec)")
            plt.ylabel("Trial #")

            plt.savefig('/home/cat/'+str(behavior)+'_average.svg')
            plt.close()
            
        stacks.append(vector_stack)
        
    allstacks.append(stacks)
    #plt.show()

  0%|          | 0/71 [00:00<?, ?it/s]

# sessions:  71


100%|██████████| 71/71 [00:10<00:00,  6.92it/s]


In [72]:
print (len(sums_array[1]))
print (sums_array[1][1])

t=np.arange(window[0], window[1],bin_width)
plt.figure()
for k in range(len(sums_array[1])):
    plt.plot(t,sums_array[1][k],label = str(k))
plt.legend()
plt.show()

22
[0.0483871  0.0483871  0.08064516 0.09677419 0.09677419 0.08064516
 0.0483871  0.11290323 0.06451613 0.09677419 0.08064516 0.0483871
 0.08064516 0.09677419 0.06451613 0.0483871  0.11290323 0.09677419
 0.14516129 0.08064516 0.12903226 0.0483871  0.08064516 0.08064516
 0.09677419 0.09677419 0.14516129 0.0483871  0.0483871  0.08064516
 0.11290323 0.11290323 0.09677419 0.06451613 0.06451613 0.08064516
 0.16129032 0.09677419 0.06451613 0.09677419 0.11290323 0.14516129
 0.06451613 0.01612903 0.06451613 0.01612903 0.0483871  0.01612903
 0.01612903 0.         0.         0.01612903 0.01612903 0.
 0.         0.11290323 0.16129032 0.08064516 0.16129032 0.09677419
 0.06451613 0.09677419 0.08064516 0.08064516 0.11290323 0.09677419
 0.16129032 0.12903226 0.16129032 0.14516129 0.09677419 0.0483871
 0.06451613 0.11290323 0.09677419 0.16129032 0.12903226 0.0483871
 0.03225806 0.11290323]


In [56]:
#############################################################
#############################################################
#############################################################
# 
print (len(sums_array[0]))
plt.figure(figsize=(5,10))
clrs=['grey','purple','gold']
names = ['left paw','right paw','licking']

for k in range(3):
    ax=plt.subplot(3,1,k+1)
    
    # 
    temp = np.array(sums_array[k])
    means = temp.mean(0)
    std = np.std(temp,0)
    t=np.arange(window[0], window[1],bin_width)

    #means+=43*k
    #std
    
    #
    plt.plot(t, means)
    ax.fill_between(t,means+std,
                     means-std,
                     color=clrs[k],
                     alpha=.2,
                     label=names[k])
    ax.set_xlim(-15,5)
    plt.legend()
plt.suptitle(animal_id)
plt.savefig('/home/cat/all_sessions_sequences'+animal_id+'.svg')
plt.close()
#plt.show()

22


In [12]:
##################################################
########## MAKE BEHAVIOR SEQUENCES VECTORS #######
##################################################
#
from tqdm import tqdm

#
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'

# 
sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  'all')
print ("# sessions: ", len(sessions))

allstacks = []
for session_id in tqdm(sessions):
    
    try:
        reward_times_sec, behavior_times_sec = get_lever_pulls_and_behaviors(session_id, root_dir, animal_id)
    except:
        continue

    # make behavivor boolean vectors
    bin_width = 0.250 # bin width in seconds
    behavior_binned = np.zeros((int(10*1//bin_width)))
    
    # 
    #print (session_id, " # of pulls: ",  behavior_times_sec[0].shape)
    names = ['left paw','right paw','licking']
    
    # loop over lever pulls:
    behaviors = [0,1,2]
    stacks = []
    for behavior in behaviors: 
        vector_stack = []
        for k in range(reward_times_sec.shape[0]):
            temp1 = reward_times_sec[k]
            temp_behavior = behavior_binned.copy()
            for p in range(behavior_times_sec[behavior].shape[0]):
                temp2 = behavior_times_sec[behavior][p]
                diff = temp1-temp2
                if diff<=10 and diff>0:
                    # find bin where it belongs
                    idx = int(behavior_binned.shape[0] - diff/bin_width)
                    #print (diff, idx)
                    temp_behavior[idx] =1

            vector_stack.append(temp_behavior)

        vector_stack = np.array(vector_stack)
        #print (vector_stack.shape)

        #
        if False:
            clrs1 = ['Greys','Purples','copper']
            plt.figure(figsize=(10,5))
            clb = plt.imshow(vector_stack,
                             aspect='auto',
                             extent=[-10,0,0,vector_stack.shape[0]],
                             cmap=clrs1[behavior]
                            )
            plt.colorbar()
            plt.xlabel("Time (sec)")
            plt.ylabel("Trial #")

            plt.savefig('/home/cat/'+str(behavior)+'.svg')
            plt.close()

        stacks.append(vector_stack)
        
    allstacks.append(stacks)
    #plt.show()

  0%|          | 0/71 [00:00<?, ?it/s]

# sessions:  71
IA1pm_Feb2_30Hz  # of pulls:  (921,)


  3%|▎         | 2/71 [00:02<01:09,  1.01s/it]

IA1pm_Feb3_30Hz  # of pulls:  (562,)


  4%|▍         | 3/71 [00:03<01:32,  1.37s/it]

IA1pm_Feb9_30Hz  # of pulls:  (3026,)


  7%|▋         | 5/71 [00:04<00:58,  1.13it/s]

KeyboardInterrupt



In [4]:
################################
############ RUN PCA ###########
################################
print (len(allstacks), len(allstacks[0]), allstacks[0][0].shape)

clrs2 = ['black','purple','khaki']
from sklearn.decomposition import PCA
a1 = []
for p in range(3):
    
    #
    temp_array =[]
    for k in range(len(allstacks)):
        temp2 = allstacks[k][p]
        temp_array.append(temp2)
    temp_array = np.vstack(temp_array)
        
    X = temp_array
    # run pca
    pca = PCA(n_components=2)
    print ("input to pca: ", X.shape)
    X_pca = pca.fit_transform(X)
    print (" pca done: ", X_pca.shape)

    fig=plt.figure(figsize=(8,8))
    plt.scatter(X_pca[:,0],
                X_pca[:,1],
                c=clrs2[p],
                #cmap = clrs1[p],
                s=100,
               # alpha=.8
               )
    plt.savefig('/home/cat/'+str(p)+'.svg')
    plt.close()
    #plt.show()
    a1.append(X)
    
# stack all feature vecotrs
X = np.vstack(np.array(a1).transpose(0,2,1)).T
print (X.shape)

# run pca
pca = PCA(n_components=2)
print ("input to pca: ", X.shape)
X_pca = pca.fit_transform(X)
print (" pca done: ", X_pca.shape)

fig=plt.figure(figsize=(8,8))
plt.scatter(X_pca[:,0],
            X_pca[:,1],
            c='blue',
            edgecolor='black',
            #cmap = clrs1[p],
            s=50,
           # alpha=.8
           )
if True:
    plt.savefig('/home/cat/sequences_AQ2.svg')
    plt.close()
else:
    plt.show()



40 3 (10, 40)
input to pca:  (2726, 40)
 pca done:  (2726, 2)
input to pca:  (2726, 40)
 pca done:  (2726, 2)
input to pca:  (2726, 40)
 pca done:  (2726, 2)
(2726, 120)
input to pca:  (2726, 120)
 pca done:  (2726, 2)
